In [7]:
import numpy as np
import pickle
import os
from sklearn.cluster import MiniBatchKMeans
from tqdm.notebook import tqdm # tqdm.notebook est plus joli sur Colab
import time


base_dir = '/content/drive/MyDrive/Colab Notebooks/Projet-ITI'
results_dir = os.path.join(base_dir, 'results')

# Inputs (ceux de l'étape 2)
descriptors_path = os.path.join(results_dir, 'all_descriptors_matrix.npy')
features_path = os.path.join(results_dir, 'image_features.pkl')

# Outputs (ceux qu'on va créer)
kmeans_model_path = os.path.join(results_dir, 'visual_vocabulary_model.pkl')
bovw_histograms_path = os.path.join(results_dir, 'bovw_histograms.npy')
bovw_image_ids_path = os.path.join(results_dir, 'bovw_image_ids.pkl')

print("✅ Configuration terminée. Chemins définis.")

✅ Configuration terminée. Chemins définis.


In [8]:
# 2. Monter Google Drive
from google.colab import drive
drive.mount('/content/drive')



print("📥 Chargement de la matrice globale des descripteurs (pour K-Means)...")
if os.path.exists(descriptors_path):
    all_descriptors = np.load(descriptors_path)
    print(f"   -> Matrice chargée. Forme : {all_descriptors.shape}")

else:
    raise FileNotFoundError(f"❌ Fichier introuvable : {descriptors_path}")


print("📥 Chargement du dictionnaire des features par image (pour Histogrammes)...")
if os.path.exists(features_path):
    with open(features_path, 'rb') as f:
        image_features = pickle.load(f)
    print(f"   -> Dictionnaire chargé. {len(image_features)} images prêtes.")
else:
    raise FileNotFoundError(f"❌ Fichier introuvable : {features_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📥 Chargement de la matrice globale des descripteurs (pour K-Means)...
   -> Matrice chargée. Forme : (4882966, 32)
📥 Chargement du dictionnaire des features par image (pour Histogrammes)...
   -> Dictionnaire chargé. 5000 images prêtes.


In [9]:
# --- 3. APPRENTISSAGE (K-MEANS) ---
K = 500  # Nombre de clusters (mots visuels) demandé par l'énoncé (200-500)

print(f"🚀 Démarrage du clustering MiniBatchKMeans (K={K})...")
print("⏳ Patientez, cela peut prendre quelques minutes...")

start_time = time.time()

# MiniBatchKMeans est beaucoup plus rapide que KMeans classique pour les gros volumes
kmeans = MiniBatchKMeans(n_clusters=K, batch_size=4096, random_state=42, n_init=3)
kmeans.fit(all_descriptors)

end_time = time.time()
print(f"✅ Dictionnaire visuel créé en {end_time - start_time:.2f} secondes.")

# Sauvegarde immédiate du modèle
with open(kmeans_model_path, 'wb') as f:
    pickle.dump(kmeans, f)
print(f"💾 Modèle K-Means sauvegardé dans : {kmeans_model_path}")

# On peut supprimer la grosse matrice pour libérer de la RAM, on n'en a plus besoin
del all_descriptors
print("🗑️ Matrice des descripteurs supprimée de la RAM pour alléger Colab.")

🚀 Démarrage du clustering MiniBatchKMeans (K=500)...
⏳ Patientez, cela peut prendre quelques minutes...
✅ Dictionnaire visuel créé en 39.72 secondes.
💾 Modèle K-Means sauvegardé dans : /content/drive/MyDrive/Colab Notebooks/Projet-ITI/results/visual_vocabulary_model.pkl
🗑️ Matrice des descripteurs supprimée de la RAM pour alléger Colab.


In [11]:
# --- 4. CRÉATION DES HISTOGRAMMES ET SAUVEGARDE (CORRIGÉ) ---
print("🏗️ Construction des histogrammes BoVW pour chaque image...")

# --- CORRECTIF CRITIQUE ---
# On force les centres du modèle à être en float64 (Double Precision)
# Cela résout l'erreur "Buffer dtype mismatch"
if kmeans.cluster_centers_.dtype != np.float64:
    print(f"⚠️ Conversion des centres de {kmeans.cluster_centers_.dtype} vers float64 pour compatibilité.")
    kmeans.cluster_centers_ = kmeans.cluster_centers_.astype(np.float64)

bovw_histograms = []
image_ids_ordered = []

# On parcourt le dictionnaire d'images
for img_id, descriptors in tqdm(image_features.items()):

    # Gestion des cas où aucun descripteur n'a été trouvé
    if descriptors is None or len(descriptors) == 0:
        histogram = np.zeros(K, dtype=np.float32)
    else:
        # --- CORRECTIF CRITIQUE ---
        # 1. On s'assure que les descripteurs sont en float64 ET contigus en mémoire
        # Scikit-learn est très strict là-dessus pour la prédiction rapide
        descriptors_double = np.ascontiguousarray(descriptors, dtype=np.float64)

        # 2. Prédiction
        words = kmeans.predict(descriptors_double)

        # 3. Compter les occurrences (créer l'histogramme)
        histogram, _ = np.histogram(words, bins=range(K + 1))

    bovw_histograms.append(histogram)
    image_ids_ordered.append(img_id)

# Conversion en matrice NumPy (X)
# Le résultat final peut rester en float32 pour économiser de la place
X_bovw = np.array(bovw_histograms, dtype=np.float32)

print(f"\n✅ Terminé ! Forme finale de X : {X_bovw.shape}")
print(f"(Cela devrait être : {len(image_features)} lignes, {K} colonnes)")

# Sauvegarde des résultats
np.save(bovw_histograms_path, X_bovw)
print(f"💾 Histogrammes sauvegardés : {bovw_histograms_path}")

with open(bovw_image_ids_path, 'wb') as f:
    pickle.dump(image_ids_ordered, f)
print(f"💾 IDs sauvegardés : {bovw_image_ids_path}")

print("\n🎉 Étape 3 Complète. Vous pouvez passer à la PCA (Étape 4).")

🏗️ Construction des histogrammes BoVW pour chaque image...
⚠️ Conversion des centres de float32 vers float64 pour compatibilité.


  0%|          | 0/5000 [00:00<?, ?it/s]


✅ Terminé ! Forme finale de X : (5000, 500)
(Cela devrait être : 5000 lignes, 500 colonnes)
💾 Histogrammes sauvegardés : /content/drive/MyDrive/Colab Notebooks/Projet-ITI/results/bovw_histograms.npy
💾 IDs sauvegardés : /content/drive/MyDrive/Colab Notebooks/Projet-ITI/results/bovw_image_ids.pkl

🎉 Étape 3 Complète. Vous pouvez passer à la PCA (Étape 4).
